In [8]:
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

import logging

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import (Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, 
                                     BatchNormalization, GlobalAveragePooling2D)
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
import datetime


In [9]:
data_dir = "../data/full/train"
val_dir = "../data/full/val"

# Augmentación para entrenamiento
datagen = ImageDataGenerator(
    rescale=1./255, 
    rotation_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    brightness_range=[0.8,1.2],
    shear_range=0.2,
    horizontal_flip=True, # Contempla manchas simétricas
)

# Generador de entrenamiento
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

datagen_val = ImageDataGenerator(
    rescale=1./255
)

val_generator = datagen_val.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

print(pd.Series(val_generator.classes).value_counts())
print(pd.Series(train_generator.classes).value_counts())

Found 8517 images belonging to 4 classes.
Found 496 images belonging to 4 classes.
2    399
3     55
1     26
0     16
Name: count, dtype: int64
2    6855
3     947
1     437
0     278
Name: count, dtype: int64


In [10]:
labels = train_generator.classes

# Calculamos los pesos
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(labels),
    y=labels
)

# Lo convertimos en diccionario para Keras
class_weights = dict(enumerate(class_weights))
print(class_weights)


{0: 7.659172661870503, 1: 4.872425629290618, 2: 0.31061269146608317, 3: 2.248416050686378}


In [11]:
next(val_generator)

(array([[[[0.57254905, 0.3529412 , 0.4039216 ],
          [0.5921569 , 0.36862746, 0.427451  ],
          [0.5882353 , 0.37647063, 0.43921572],
          ...,
          [0.7176471 , 0.5647059 , 0.63529414],
          [0.7294118 , 0.5686275 , 0.6392157 ],
          [0.7176471 , 0.5568628 , 0.627451  ]],
 
         [[0.5647059 , 0.32156864, 0.3803922 ],
          [0.59607846, 0.38431376, 0.43921572],
          [0.62352943, 0.40000004, 0.45882356],
          ...,
          [0.72156864, 0.5568628 , 0.63529414],
          [0.72156864, 0.5529412 , 0.627451  ],
          [0.7058824 , 0.5372549 , 0.6039216 ]],
 
         [[0.54509807, 0.3019608 , 0.3529412 ],
          [0.5921569 , 0.38823533, 0.43921572],
          [0.6313726 , 0.41960788, 0.47450984],
          ...,
          [0.7176471 , 0.5568628 , 0.627451  ],
          [0.7176471 , 0.54901963, 0.6156863 ],
          [0.7019608 , 0.5372549 , 0.5921569 ]],
 
         ...,
 
         [[0.654902  , 0.48235297, 0.54901963],
          [0.65882

## Modelo

In [12]:
def clasificador_binario(input_shape=(224,224,3), lr=1e-3):
    entrada = Input(shape=input_shape, name='entrada_imagen')

    # Bloque 1
    x = Conv2D(32, (3,3), activation='relu', padding='same', kernel_regularizer=l2(1e-4))(entrada)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2,2))(x)

    # Bloque 2
    x = Conv2D(64, (3,3), activation='relu', padding='same', kernel_regularizer=l2(1e-4))(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2,2))(x)

    # Bloque 3
    x = Conv2D(128, (3,3), activation='relu', padding='same', kernel_regularizer=l2(1e-4))(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2,2))(x)

    # Bloque 4 (extra para más capacidad)
    x = Conv2D(256, (3,3), activation='relu', padding='same', kernel_regularizer=l2(1e-4))(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2,2))(x)

    # Global pooling en lugar de Flatten (reduce parámetros)
    x = GlobalAveragePooling2D()(x)

    # Capa densa
    x = Dense(128, activation='relu', kernel_regularizer=l2(1e-4))(x)
    x = Dropout(0.2)(x)

    salida = Dense(4, activation='softmax', name='salida_clases')(x)

    modelo = Model(inputs=entrada, outputs=salida, name='cnn_mejorada')
    modelo.compile(optimizer=Adam(learning_rate=lr),
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])
    return modelo



In [13]:
import tensorflow as tf

# Desactiva todas las GPUs
tf.config.set_visible_devices([], 'GPU')


In [14]:
# Ajustar nivel de logging de TensorFlow
logging.getLogger("tensorflow").setLevel(logging.ERROR)

modelo = clasificador_binario()  

history = modelo.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    class_weight=class_weights
)


Epoch 1/20


2025-11-29 13:16:41.254867: I external/local_xla/xla/service/service.cc:163] XLA service 0x7f7268015530 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2025-11-29 13:16:41.254886: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): Host, Default Version
2025-11-29 13:16:41.356844: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1764418602.912822   77518 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2025-11-29 13:16:42.915768: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 2667125544 exceeds 10% of free system memory.


  1/267 ━━━━━━━━━━━━━━━━━━━━ 34:39 8s/step - accuracy: 0.5000 - loss: 1.3797

2025-11-29 13:16:45.149660: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 2667125544 exceeds 10% of free system memory.


  2/267 ━━━━━━━━━━━━━━━━━━━━ 7:59 2s/step - accuracy: 0.4922 - loss: 1.6559 

...

  3/267 ━━━━━━━━━━━━━━━━━━━━ 8:03 2s/step - accuracy: 0.5052 - loss: 1.7011

2025-11-29 13:16:48.816259: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 2667125544 exceeds 10% of free system memory.


  4/267 ━━━━━━━━━━━━━━━━━━━━ 8:06 2s/step - accuracy: 0.5098 - loss: 1.7823

2025-11-29 13:16:50.694231: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 2667125544 exceeds 10% of free system memory.


267/267 ━━━━━━━━━━━━━━━━━━━━ 519s 2s/step - accuracy: 0.4552 - loss: 1.2396 - val_accuracy: 0.6976 - val_loss: 1.0997
Epoch 2/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 514s 2s/step - accuracy: 0.4974 - loss: 1.1203 - val_accuracy: 0.3427 - val_loss: 1.5615
Epoch 3/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 516s 2s/step - accuracy: 0.5194 - loss: 1.0869 - val_accuracy: 0.6996 - val_loss: 0.7158
Epoch 4/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 514s 2s/step - accuracy: 0.5306 - loss: 1.0505 - val_accuracy: 0.6230 - val_loss: 0.8703
Epoch 5/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 515s 2s/step - accuracy: 0.5540 - loss: 1.0361 - val_accuracy: 0.3931 - val_loss: 1.2568
Epoch 6/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 515s 2s/step - accuracy: 0.5653 - loss: 1.0251 - val_accuracy: 0.6169 - val_loss: 0.9488
Epoch 7/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 517s 2s/step - accuracy: 0.5575 - loss: 1.0205 - val_accuracy: 0.4718 - val_loss: 1.1045
Epoch 8/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 514s 2s/step - accuracy: 0.5645 - loss: 1.0077 - val_accuracy: 0.663

In [ ]:
# Current timestamp
timestamp = datetime.datetime.now().strftime("%m_%d_h%H_%M")

# Carpeta donde guardar
save_dir = "../models/classifier"
os.makedirs(save_dir, exist_ok=True)

modelo.save(f"../models/full/full_model_{timestamp}.keras")

In [16]:
modelo.summary()

Model: "cnn_mejorada"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ entrada_imagen (InputLayer)     │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 224, 224, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 28, 28, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 28, 28, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 14, 14, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 256)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ salida_clases (Dense)           │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,269,326 (4.84 MB)

 Trainable params: 422,788 (1.61 MB)

 Non-trainable params: 960 (3.75 KB)

 Optimizer params: 845,578 (3.23 MB)